# Test Streaming Follow-up API

This notebook tests the streaming follow-up API endpoint for the dermatology LLM assistant.

## Setup

First, set your Modal endpoint URL and test data.


In [2]:
import os
import requests
import json
from typing import Optional

# Set your Modal endpoint URL here
# Format: https://{username}--{app-name}-dermatologyllm-ask-followup-stream.modal.run
STREAMING_URL = os.getenv(
    "MODAL_STREAMING_URL",
    "https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-ask-fo-8013b2.modal.run/"
)

print(f"Streaming endpoint: {STREAMING_URL}")


Streaming endpoint: https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-ask-fo-8013b2.modal.run/


In [20]:
import pandas as pd
x = pd.read_csv("/Users/tk20/Downloads/dataset_v1_metadata_all_harmonized.csv")

/var/folders/0k/wz8nbvps6g565rr_vdwxl5w80000gn/T/ipykernel_72899/1145721198.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  x = pd.read_csv("/Users/tk20/Downloads/dataset_v1_metadata_all_harmonized.csv")


In [23]:
x['label'].unique()

array(['melanoma', 'cutaneous-lymphoma', 'squamous-cell-carcinoma',
       'basal-cell-carcinoma', 'benign-nevus', 'seborrheic-keratosis',
       'benign-keratosis', 'hyperpigmentation', 'lipoma',
       'granuloma-annulare', 'wart-hpv', 'skin-tag', 'trauma',
       'epidermal-cyst', 'dermatofibroma', 'hair-follicle-tumor',
       'pyogenic-granuloma', 'neurofibroma', 'sweat-gland-tumor',
       'fungal-infection', 'lymphocytic-infiltrations', 'prurigo',
       'kaposi-sarcoma', 'scar-fibrosis', 'vascular-tumor',
       'metastatic-carcinoma', 'folliculitis', 'dysplastic-nevus',
       'xanthogranuloma', 'eczema-dermatitis', 'acne', 'xanthoma',
       'viral-infection', 'scleroderma-morphea', 'actinic-keratosis',
       'dermatomyositis', 'graft-vs-host-disease', 'benign-hyperplasia',
       'sebaceous-carcinoma', 'lentigo-solar', 'clear-cell-acanthoma',
       'bacterial-infection', 'melanotic-macule', 'photodermatitis',
       'neutrophilic-dermatoses', 'genodermatosis', 'pityriasis'

## Test Data

Define your test inputs:


In [3]:
# Example initial answer from the LLM
initial_answer = """Based on the image analysis, I can see a skin lesion that appears to be:
- Irregular borders
- Multiple colors (brown, black, tan)
- Asymmetric shape
- Diameter approximately 6mm

These characteristics suggest this may be a concerning lesion that requires further evaluation by a dermatologist."""

# Your follow-up question
question = "What should I do next?"

# Optional: conversation history (list of previous questions)
conversation_history = []

print(f"Question: {question}")
print(f"Initial answer length: {len(initial_answer)} characters")
print(f"Conversation history: {len(conversation_history)} previous questions")


Question: What should I do next?
Initial answer length: 294 characters
Conversation history: 0 previous questions


In [4]:
# Prepare payload
payload = {
    "initial_answer": initial_answer,
    "question": question,
    "conversation_history": conversation_history,
}

print("Making streaming request...")
print("=" * 60)

try:
    # Make streaming request
    response = requests.post(
        STREAMING_URL,
        json=payload,
        stream=True,
        timeout=300,  # 5 minute timeout
    )
    response.raise_for_status()
    
    # Process streaming response
    full_answer_parts = []
    chunk_count = 0
    
    # Iterate over lines (newline-delimited JSON)
    for line in response.iter_lines(decode_unicode=True):
        if not line:
            continue
        
        chunk_count += 1
        text_chunk = None
        
        # Try to parse as JSON first
        try:
            data = json.loads(line)
            # The endpoint sends JSON-encoded strings, so data will be a string
            if isinstance(data, str):
                text_chunk = data
            elif isinstance(data, dict):
                # Handle dict format if endpoint changes
                text_chunk = (
                    data.get("delta")
                    or data.get("text")
                    or data.get("answer_chunk")
                    or data.get("answer")
                )
        except json.JSONDecodeError:
            # If not JSON, treat as plain text
            text_chunk = line
        
        if text_chunk:
            full_answer_parts.append(text_chunk)
            # Print chunk as it arrives
            print(text_chunk, end="", flush=True)
    
    print("\n" + "=" * 60)
    print(f"\n✅ Streaming complete!")
    print(f"Total chunks received: {chunk_count}")
    print(f"Full answer length: {len(''.join(full_answer_parts))} characters")
    
    full_answer = "".join(full_answer_parts).strip()
    
except requests.exceptions.RequestException as e:
    print(f"\n❌ Error calling streaming API: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response status: {e.response.status_code}")
        print(f"Response body: {e.response.text}")
    raise


Making streaming request...
Okay, based on what we saw in the picture, with those irregular borders, multiple colors, asymmetry, and size, the most important next step is to get it checked out by a dermatologist as soon as possible.

Remember, I'm an AI assistant and can't give medical advice, but those features are ones that doctors look for when evaluating moles. A dermatologist has the tools and expertise to examine it properly, determine what it is, and discuss the best course of action. They might recommend monitoring it, removing it for a biopsy, or another approach.

Don't delay in making that appointment – early detection is key for many skin conditions. Let me know if you have any other general questions while you wait!

✅ Streaming complete!
Total chunks received: 121
Full answer length: 710 characters


In [5]:
# Example with conversation history
initial_answer = """Based on the image analysis, I can see a skin lesion that appears to be:
- Irregular borders
- Multiple colors (brown, black, tan)
- Asymmetric shape
- Diameter approximately 6mm

These characteristics suggest this may be a concerning lesion that requires further evaluation by a dermatologist."""

# Previous questions in the conversation
conversation_history = [
    "What does this look like?",
    "Is it dangerous?",
]

# Current question
question = "What are the treatment options?"

payload = {
    "initial_answer": initial_answer,
    "question": question,
    "conversation_history": conversation_history,
}

print("Testing with conversation history...")
print(f"Previous questions: {len(conversation_history)}")
print(f"Current question: {question}")
print("=" * 60)

try:
    response = requests.post(STREAMING_URL, json=payload, stream=True, timeout=300)
    response.raise_for_status()
    
    full_answer_parts = []
    for line in response.iter_lines(decode_unicode=True):
        if not line:
            continue
        try:
            data = json.loads(line)
            if isinstance(data, str):
                text_chunk = data
            elif isinstance(data, dict):
                text_chunk = data.get("delta") or data.get("text") or data.get("answer")
            else:
                text_chunk = None
        except json.JSONDecodeError:
            text_chunk = line
        
        if text_chunk:
            full_answer_parts.append(text_chunk)
            print(text_chunk, end="", flush=True)
    
    print("\n" + "=" * 60)
    print("✅ Test with history complete!")
    
except Exception as e:
    print(f"\n❌ Error: {e}")


Testing with conversation history...
Previous questions: 2
Current question: What are the treatment options?
Okay, let's talk about treatment options. Since the initial analysis suggested this lesion has some concerning features, the *first* and most important step is to get it checked out by a dermatologist. They'll likely want to do a biopsy to get a definitive diagnosis.

If it turns out to be something like melanoma, treatment depends heavily on the stage. Early-stage melanomas are often treated with surgical excision – basically, the doctor removes the lesion and a small margin of surrounding healthy skin. For more advanced cases, other treatments like immunotherapy, targeted therapy, radiation, or chemotherapy might be considered.

If it's something less serious, like a benign mole or seborrheic keratosis, treatment might involve simple removal for cosmetic reasons or if it's irritated, or sometimes just monitoring it.

Remember, this is just general information. A dermatologist 

## Test APIManager chat_message (Streaming)

Test the `chat_message` function from APIManager that uses the streaming LLM followup endpoint:


In [6]:
import sys
from pathlib import Path

# Add the api_manager path to sys.path if needed
# Assuming notebook is in src/llm/, we need to go to src/frontend-react/python
current_dir = Path.cwd()
if current_dir.name == "llm":
    # We're in src/llm/
    api_manager_path = current_dir.parent / "frontend-react" / "python"
else:
    # Try relative to project root
    api_manager_path = Path.cwd() / "src" / "frontend-react" / "python"

if str(api_manager_path) not in sys.path:
    sys.path.insert(0, str(api_manager_path))

from api_manager import APIManager

# Set up a test case ID
test_case_id = "case_test_streaming"

# Initialize APIManager
api_manager = APIManager(case_id=test_case_id, dummy=False)

print(f"Initialized APIManager for case: {test_case_id}")
print(f"LLM Followup URL: {api_manager.llm_followup_url}")


/Users/tk20/Desktop/APCOMP215/ac215_Group_127_2473879/src/llm/case_test_streaming
No case history file found for case case_test_streaming, using default.
No demographics file found for case case_test_streaming, using default.


Initialized APIManager for case: case_test_streaming
LLM Followup URL: https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-ask-fo-8013b2.modal.run/


### Setup: Create initial conversation entry (if needed)

If you don't have an existing conversation, create a minimal one:


In [7]:
# Check if conversation exists, if not create a minimal one
import json
from datetime import datetime, timezone

conversation_file = api_manager.conversation_file

if not conversation_file.exists() or not json.load(open(conversation_file)):
    print("Creating initial conversation entry...")
    
    # Create a minimal initial message
    initial_message = {
        "user": {
            "message": "I have a skin lesion on my arm",
            "timestamp": datetime.now(timezone.utc).isoformat()
        },
        "llm": {
            "message": "Based on the image analysis, I can see a skin lesion that appears to have irregular borders, multiple colors (brown, black, tan), asymmetric shape, and a diameter of approximately 6mm. These characteristics suggest this may be a concerning lesion that requires further evaluation by a dermatologist.",
            "timestamp": datetime.now(timezone.utc).isoformat()
        }
    }
    
    # Save initial conversation
    with open(conversation_file, 'w') as f:
        json.dump([initial_message], f, indent=2)
    
    print("✓ Initial conversation created")
else:
    print("✓ Conversation history already exists")
    with open(conversation_file, 'r') as f:
        conv = json.load(f)
        print(f"  Found {len(conv)} conversation entries")


✓ Conversation history already exists
  Found 3 conversation entries


### Test chat_message with streaming

Call `chat_message` and stream the response:


In [8]:
# Define callback to handle streaming chunks
full_response_chunks = []

def on_chunk(chunk: str):
    """Callback function that receives streaming chunks."""
    full_response_chunks.append(chunk)
    # Print chunk as it arrives (without newline for inline updates)
    print(chunk, end="", flush=True)

# Test question
test_question = "What should I do next?"

print(f"Question: {test_question}")
print("=" * 60)
print("Streaming response:")
print("=" * 60)

try:
    # Call chat_message with streaming callback
    result = api_manager.chat_message(
        user_query=test_question,
        user_timestamp=datetime.now(timezone.utc).isoformat(),
        on_chunk=on_chunk  # Streaming callback
    )
    
    print("\n" + "=" * 60)
    print("✅ Streaming complete!")
    print("=" * 60)
    
    # Display results
    print(f"\nFull answer: {result['answer']}")
    print(f"\nConversation history length: {len(result.get('conversation_history', []))}")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


Processing chat message for case case_test_streaming...
  → Loading conversation history...
  → Calling LLM for follow-up answer...
 Calling LLM followup API (streaming only)...


Question: What should I do next?
Streaming response:
Okay, let's talk about next steps. Since the lesion has several characteristics that warrant a closer look, the most important thing is to schedule an appointment with a dermatologist as soon as possible. They can perform a thorough examination, potentially using tools like a dermatoscope, to get a better understanding of what's going on.

Based on their assessment, they might recommend monitoring the spot, taking a biopsy to examine the cells under a microscope, or removing the lesion entirely. Don't worry too much right now, but please do prioritize getting it checked out. Early detection is key for many skin conditions, and a dermatologist is the best person to give you a definitive diagnosis and appropriate treatment plan. Let me know if you have any other questions while you're waiting for your appointment!

  → Saving conversation...
✓ Chat message processed for case case_test_streaming



✅ Streaming complete!

Full answer: Okay, let's talk about next steps. Since the lesion has several characteristics that warrant a closer look, the most important thing is to schedule an appointment with a dermatologist as soon as possible. They can perform a thorough examination, potentially using tools like a dermatoscope, to get a better understanding of what's going on.

Based on their assessment, they might recommend monitoring the spot, taking a biopsy to examine the cells under a microscope, or removing the lesion entirely. Don't worry too much right now, but please do prioritize getting it checked out. Early detection is key for many skin conditions, and a dermatologist is the best person to give you a definitive diagnosis and appropriate treatment plan. Let me know if you have any other questions while you're waiting for your appointment!

Conversation history length: 2


### Verify: Check accumulated chunks

Verify that all chunks were received:


In [9]:
# Verify chunks
if full_response_chunks:
    accumulated = "".join(full_response_chunks)
    print(f"Total chunks received: {len(full_response_chunks)}")
    print(f"Total characters: {len(accumulated)}")
    print(f"\nAccumulated response:\n{accumulated}")
else:
    print("No chunks received")


Total chunks received: 133
Total characters: 823

Accumulated response:
Okay, let's talk about next steps. Since the lesion has several characteristics that warrant a closer look, the most important thing is to schedule an appointment with a dermatologist as soon as possible. They can perform a thorough examination, potentially using tools like a dermatoscope, to get a better understanding of what's going on.

Based on their assessment, they might recommend monitoring the spot, taking a biopsy to examine the cells under a microscope, or removing the lesion entirely. Don't worry too much right now, but please do prioritize getting it checked out. Early detection is key for many skin conditions, and a dermatologist is the best person to give you a definitive diagnosis and appropriate treatment plan. Let me know if you have any other questions while you're waiting for your appointment!


## Test explain_stream Endpoint

Test the streaming explanation endpoint that takes a prompt and streams the response:


In [10]:
# Set your explain_stream endpoint URL
# Format: https://{username}--{app-name}-dermatologyllm-explain-stream.modal.run
EXPLAIN_STREAM_URL = os.getenv(
    "MODAL_EXPLAIN_STREAM_URL",
    "https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-explai-0d573f.modal.run"
)

print(f"Explain stream endpoint: {EXPLAIN_STREAM_URL}")


Explain stream endpoint: https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-explai-0d573f.modal.run


In [16]:
# Sample predictions (disease probabilities)
predictions = {
    "eczema": 0.78,
    "contact_dermatitis": 0.15,
    "psoriasis": 0.04,
    "tinea_corporis": 0.02,
    "seborrheic_dermatitis": 0.01,
}

# Sample metadata
metadata = {
    "user_input": "I have a red, itchy rash on my arm that appeared a few days ago.",
    "cv_analysis": {
        "area": 8.4,
        "color_profile": {
            "average_Lab": [67.2, 18.4, 9.3],
            "redness_index": 0.34,
            "texture_contrast": 0.12
        },
        "boundary_irregularity": 0.23,
        "symmetry_score": 0.78,
    },
    "history": {}  # Empty history for this test
}

# Prepare payload (same format as non-streaming explain endpoint)
payload = {
    "predictions": predictions,
    "metadata": metadata,
}

print("Making streaming request to explain_stream...")
print(f"Top prediction: {max(predictions.items(), key=lambda x: x[1])}")
print(f"User input: {metadata['user_input']}")
print("=" * 60)
print("Streaming response:")
print("=" * 60)

try:
    # Make streaming request
    response = requests.post(
        EXPLAIN_STREAM_URL,
        json=payload,
        stream=True,
        timeout=300,  # 5 minute timeout
    )
    response.raise_for_status()
    
    # Process streaming response
    full_answer_parts = []
    chunk_count = 0
    
    # Iterate over lines (newline-delimited JSON)
    for line in response.iter_lines(decode_unicode=True):
        if not line:
            continue
        
        chunk_count += 1
        
        # Parse JSON line
        try:
            data = json.loads(line)
            # The endpoint sends {"delta": "text_chunk"}
            if isinstance(data, dict):
                text_chunk = data.get("delta")
            elif isinstance(data, str):
                # Fallback if format changes
                text_chunk = data
            else:
                text_chunk = None
        except json.JSONDecodeError:
            # If not JSON, treat as plain text
            text_chunk = line
        
        if text_chunk:
            full_answer_parts.append(text_chunk)
            # Print chunk as it arrives
            print(text_chunk, end="", flush=True)
    
    print("\n" + "=" * 60)
    print(f"\n✅ Streaming complete!")
    print(f"Total chunks received: {chunk_count}")
    print(f"Full answer length: {len(''.join(full_answer_parts))} characters")
    
    full_answer = "".join(full_answer_parts).strip()
    
except requests.exceptions.RequestException as e:
    print(f"\n❌ Error calling streaming API: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response status: {e.response.status_code}")
        print(f"Response body: {e.response.text}")
    raise


Making streaming request to explain_stream...
Top prediction: ('eczema', 0.78)
User input: I have a red, itchy rash on my arm that appeared a few days ago.
Streaming response:
Hi there! It sounds like you're dealing with a bothersome rash, and I'm here to help shed some light on what might be going on.

**Most likely condition**
Based on what you've described and the image analysis, this looks most like eczema. It's a common condition that causes skin to become red and itchy.

**What the image + history show**
The image shows a red rash covering a noticeable area of your arm, and the redness level is moderate. Since you mentioned it appeared just a few days ago, it seems like a relatively recent flare-up.

**Common causes**
Eczema often pops up when your skin's natural barrier is disrupted. This can happen due to things like dry air, harsh soaps, or even stress. Sometimes, it can also be triggered by contact with certain materials or allergens.

**Home care**
To help soothe the itch an

In [17]:
# Display the full explanation
if 'full_answer' in locals():
    print("Full Explanation:")
    print("-" * 60)
    print(full_answer)
    print("-" * 60)
else:
    print("No answer received. Check the streaming request above.")


Full Explanation:
------------------------------------------------------------
Hi there! It sounds like you're dealing with a bothersome rash, and I'm here to help shed some light on what might be going on.

**Most likely condition**
Based on what you've described and the image analysis, this looks most like eczema. It's a common condition that causes skin to become red and itchy.

**What the image + history show**
The image shows a red rash covering a noticeable area of your arm, and the redness level is moderate. Since you mentioned it appeared just a few days ago, it seems like a relatively recent flare-up.

**Common causes**
Eczema often pops up when your skin's natural barrier is disrupted. This can happen due to things like dry air, harsh soaps, or even stress. Sometimes, it can also be triggered by contact with certain materials or allergens.

**Home care**
To help soothe the itch and redness, try applying a gentle, fragrance-free moisturizer several times a day, especially afte

## Test APIManager _call_llm_explain (Streaming)

Test the `_call_llm_explain` function from APIManager that uses the streaming explain endpoint:


In [18]:
# Use the same APIManager instance from earlier, or create a new one
# If api_manager is not defined, create it
if 'api_manager' not in locals():
    import sys
    from pathlib import Path
    
    current_dir = Path.cwd()
    if current_dir.name == "llm":
        api_manager_path = current_dir.parent / "frontend-react" / "python"
    else:
        api_manager_path = Path.cwd() / "src" / "frontend-react" / "python"
    
    if str(api_manager_path) not in sys.path:
        sys.path.insert(0, str(api_manager_path))
    
    from api_manager import APIManager
    api_manager = APIManager(case_id="case_test_explain", dummy=False)

print(f"Using APIManager for case: {api_manager.case_id}")


Using APIManager for case: case_test_streaming


### Test _call_llm_explain with streaming

Create sample predictions and metadata, then call the streaming explain function:


In [19]:
# Define callback to handle streaming chunks
explain_chunks = []

def on_chunk_explain(chunk: str):
    """Callback function that receives streaming chunks from explain."""
    explain_chunks.append(chunk)
    # Print chunk as it arrives (without newline for inline updates)
    print(chunk, end="", flush=True)

# Sample predictions (disease probabilities)
predictions = {
    "eczema": 0.78,
    "contact_dermatitis": 0.15,
    "psoriasis": 0.04,
    "tinea_corporis": 0.02,
    "seborrheic_dermatitis": 0.01,
}

# Sample metadata
metadata = {
    "user_input": "I have a red, itchy rash on my arm that appeared a few days ago.",
    "cv_analysis": {
        "area": 8.4,
        "color_profile": {
            "average_Lab": [67.2, 18.4, 9.3],
            "redness_index": 0.34,
            "texture_contrast": 0.12
        },
        "boundary_irregularity": 0.23,
        "symmetry_score": 0.78,
    },
    "history": {}  # Empty history for this test
}

print("Test Data:")
print(f"Predictions: {list(predictions.keys())}")
print(f"Top prediction: {max(predictions.items(), key=lambda x: x[1])}")
print("=" * 60)
print("Streaming explanation:")
print("=" * 60)

try:
    # Call _call_llm_explain with streaming callback
    result, timestamp = api_manager._call_llm_explain(
        predictions=predictions,
        metadata=metadata,
        on_chunk=on_chunk_explain  # Streaming callback
    )
    
    print("\n" + "=" * 60)
    print("✅ Streaming complete!")
    print("=" * 60)
    
    # Display results
    print(f"\nFull answer length: {len(result.get('answer', ''))} characters")
    print(f"Timestamp: {timestamp}")
    print(f"\nAnswer preview (first 200 chars):")
    print(result.get('answer', '')[:200] + "...")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


 Calling LLM explain API (streaming)...


Test Data:
Predictions: ['eczema', 'contact_dermatitis', 'psoriasis', 'tinea_corporis', 'seborrheic_dermatitis']
Top prediction: ('eczema', 0.78)
Streaming explanation:
Hi there! It sounds like you're dealing with something uncomfortable on your arm.

Based on what you've described and the image, the most likely condition is eczema. It appears as a red, itchy rash, and the redness level is relatively low, which fits with typical eczema presentations.

The image and your history show a red rash on your arm that's causing itchiness. Since it appeared just a few days ago, it seems to be a relatively recent development.

Eczema often pops up because of dry skin, irritation from things like soaps or detergents, or sometimes even stress or allergies. It's quite common for skin to react this way to different triggers.

To help soothe it at home, try applying a gentle, fragrance-free moisturizer several times a day, especially after bathing, to keep the skin hydrated. You could also try a cool

### Verify: Check accumulated chunks

Verify that all chunks were received correctly:
